# Test registration on velodyne scans

In [ ]:
%matplotlib widget
import numpy as np
import os
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from copy import deepcopy
import open3d as o3d

import planeslam.io as io
from planeslam.scan import velo_pc_to_scan
from planeslam.general import NED_to_ENU, trajectory_plot_trace, pc_plot_trace
from planeslam.geometry.util import quat_to_R
from planeslam.registration import get_correspondences
from planeslam.point_cloud import velo_preprocess

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
# Read in point cloud data
pcpath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_12_2022', 'flightroom', 'run_1', 'pcs')
PCs = []
#len(os.listdir(pcpath))
select_idxs = np.arange(0, 200, 2)
for i in select_idxs:  
    filename = pcpath+'\pc_'+str(i)+'.npy'
    PC = np.load(filename)
    PCs.append(PC)

In [ ]:
# Read in pose data
posepath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_12_2022', 'flightroom', 'run_1', 'poses')
poses = []
for i in select_idxs:  
    filename = posepath+'\pose_'+str(i)+'.npy'
    pose = np.load(filename)
    poses.append(pose)

In [ ]:
# Plot ground-truth trajectory
rover_positions = np.asarray(poses)[:,:3]
gt_traj_trace = go.Scatter3d(x=rover_positions[:,0], y=rover_positions[:,1], z=rover_positions[:,2], hovertext=np.arange(len(rover_positions)))
fig = go.Figure(data=gt_traj_trace)
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Extract scans
num_scans = len(PCs)
scans = num_scans * [None]
# scans_transformed = num_scans * [None]
for i in range(num_scans):
    print(i)
    P = velo_preprocess(PCs[i], poses[i])
    scans[i] = velo_pc_to_scan(P)
    scans[i].remove_small_planes(area_thresh=0.1)
    scans[i].reduce_inside(p2p_dist_thresh=0.1)
    #scans[i].transform(quat_to_R(poses[s][3:]), poses[s][:3])

In [ ]:
# Plot 2 scans
source_idx = 44
target_idx = 43
source = scans[source_idx]
target = scans[target_idx]

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

for t in source.plot_trace(show_normals=False):
    fig.add_trace(t, row=1, col=1)

for t in target.plot_trace(show_normals=False):
    fig.add_trace(t, row=1, col=2)

fig.update_layout(width=1600, height=700, scene=dict(aspectmode='data'), scene2=dict(aspectmode='data'))
fig.show()

correspondences = get_correspondences(source, target)
print(correspondences)

In [ ]:
from planeslam.registration import robust_GN_register

R_hat, t_hat = robust_GN_register(source, target, t_loss_thresh=0.1)

print(R_hat)
print(t_hat)

In [ ]:
from planeslam.geometry.util import quat_to_R

R_1 = quat_to_R(poses[target_idx][3:])
R_2 = quat_to_R(poses[source_idx][3:])
R_gt = R_2 @ R_1.T

t_gt = poses[source_idx][:3] - poses[target_idx][:3]
print(R_gt)
print(t_gt)

In [ ]:
from planeslam.registration import robust_GN_register

R_abs = quat_to_R(poses[0][3:])
t_abs = poses[0][:3].copy()
traj_est = np.zeros((len(scans), 3))
traj_est[0] = t_abs
avg_runtime = 0

for i in range(1, len(scans)):
    print("i = ", i)
    start_time = time.time()
    R_hat, t_hat = robust_GN_register(scans[i], scans[i-1], t_loss_thresh=0.1)
    t_abs += (R_abs @ t_hat).flatten()
    R_abs = R_hat @ R_abs
    avg_runtime += time.time() - start_time
    traj_est[i] = t_abs

    R_1 = quat_to_R(poses[i-1][3:])
    R_2 = quat_to_R(poses[i][3:])
    R_gt = R_2 @ R_1.T

    t_gt = poses[i][:3] - poses[i-1][:3]
    print("R_hat: ", R_hat)
    print("R_gt: ", R_gt)
    print("t_hat: ", t_hat.flatten())
    print("t_gt: ", t_gt)

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

In [ ]:
# Plot trajectories
est_traj_trace = go.Scatter3d(x=traj_est[:,0], y=traj_est[:,1], z=traj_est[:,2], hovertext=np.arange(len(traj_est)))
fig = go.Figure(data=[gt_traj_trace, est_traj_trace])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()